In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import tqdm
import re

In [9]:
def load_data(file_path, sample_size=10000, random_sample=True):
    if sample_size is None or sample_size <= 0:
        df = pd.read_csv(file_path)
        print(f"Loaded full dataset with shape: {df.shape}")
        return df

    if random_sample:
        df = pd.read_csv(file_path).sample(sample_size)
    else:
        df = pd.read_csv(file_path, nrows=sample_size)

    print(f"Loaded sample data with shape: {df.shape}")
    print(f"Sample contains data from {df['date'].min()} to {df['date'].max()}")
    print(f"Sample includes {df['store'].nunique()} unique stores and {df['category_name'].nunique()} product categories")

    return df

def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    if 'store_location' in df.columns and isinstance(df['store_location'].iloc[0], str):
        try:
            df['store_location'] = df['store_location'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

            df['longitude'] = df['store_location'].apply(lambda x: x.get('coordinates', [None, None])[0] if isinstance(x, dict) else None)
            df['latitude'] = df['store_location'].apply(lambda x: x.get('coordinates', [None, None])[1] if isinstance(x, dict) else None)
        except:
            print("Warning: Could not parse store_location JSON.")

    for col in df.select_dtypes(include=['number']).columns:
        df[col] = df[col].fillna(0)

    return df

In [3]:
def aggregate_store_product_sales(df, time_period='M'):
    df['time_period'] = df['date'].dt.to_period(time_period)

    agg_sales = df.groupby(['store', 'name', 'category_name', 'time_period']).agg({
        'sale_dollars': 'sum',
        'sale_bottles': 'sum',
        'sale_liters': 'sum',
        'sale_gallons': 'sum'
    }).reset_index()

    agg_sales['time_period'] = agg_sales['time_period'].dt.to_timestamp()

    return agg_sales

def create_store_product_features(agg_sales):
    features_list = []

    store_products = agg_sales[['store', 'name', 'category_name']].drop_duplicates()

    for _, sp in store_products.iterrows():
        store_id = sp['store']
        store_name = sp['name']
        product = sp['category_name']

        sp_data = agg_sales[
            (agg_sales['store'] == store_id) &
            (agg_sales['category_name'] == product)
        ].sort_values('time_period')

        if len(sp_data) >= 3:
            sales_values = sp_data['sale_dollars'].values
            sales_bottles = sp_data['sale_bottles'].values
            time_periods = sp_data['time_period'].values

            avg_sales = np.mean(sales_values)
            max_sales = np.max(sales_values)
            min_sales = np.min(sales_values)
            last_sales = sales_values[-1]
            last_3_avg = np.mean(sales_values[-3:]) if len(sales_values) >= 3 else last_sales

            x = np.arange(len(sales_values))
            slope, intercept = np.polyfit(x, sales_values, 1)

            seasonality = 0
            if len(sales_values) >= 12:
                year_diff = []
                for i in range(len(sales_values) - 12):
                    year_diff.append(sales_values[i + 12] - sales_values[i])
                seasonality = np.mean(year_diff) if year_diff else 0

            if len(sales_values) >= 3:
                recent_change = (sales_values[-1] - sales_values[-3]) / (sales_values[-3] if sales_values[-3] > 0 else 1)
            else:
                recent_change = 0

            is_declining = slope < 0 and last_sales < avg_sales

            features = {
                'store_id': store_id,
                'store_name': store_name,
                'product': product,
                'num_periods': len(sales_values),
                'avg_sales': avg_sales,
                'max_sales': max_sales,
                'min_sales': min_sales,
                'last_sales': last_sales,
                'last_3_avg': last_3_avg,
                'sales_slope': slope,
                'sales_intercept': intercept,
                'seasonality': seasonality,
                'recent_change_rate': recent_change,
                'last_date': time_periods[-1],
                'first_date': time_periods[0],
                'is_declining_basic': is_declining,
                'total_bottles': np.sum(sales_bottles)
            }

            features_list.append(features)

    features_df = pd.DataFrame(features_list)

    return features_df

In [4]:
def load_qwen_model():
    model_name = "Qwen/Qwen2.5-0.5B-Instruct"

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer

In [5]:
def create_llm_prompt_for_prediction(features_row, additional_store_info=None):
    first_date = pd.to_datetime(features_row['first_date']).strftime('%Y-%m-%d')
    last_date = pd.to_datetime(features_row['last_date']).strftime('%Y-%m-%d')

    prompt = f"""
    Sales Analysis Task:

    Store Information:
    - Store ID: {features_row['store_id']}
    - Store Name: {features_row['store_name']}
    {additional_store_info if additional_store_info else ''}

    Product Category: {features_row['product']}

    Sales Data:
    - Time Period: {first_date} to {last_date}
    - Number of data points: {features_row['num_periods']}
    - Average sales: ${features_row['avg_sales']:.2f}
    - Last recorded sales: ${features_row['last_sales']:.2f}
    - Recent 3-period average: ${features_row['last_3_avg']:.2f}
    - Sales trend slope: {features_row['sales_slope']:.4f} (negative means declining)
    - Recent change rate: {features_row['recent_change_rate']:.2f}
    - Total bottles sold: {features_row['total_bottles']}

    Task: Based on the data above, please analyze:
    1. Will sales for this product at this store continue to decline? (Yes/No)
    2. What is your confidence level in this prediction? (Low/Medium/High)

    The analysis should be formatted clearly with "Prediction: " followed by Yes or No, and "Confidence: " followed by Low, Medium, or High.

    Then provide a brief explanation of your reasoning in 2-3 sentences.
    """

    return prompt

def get_llm_prediction(model, tokenizer, features_row, additional_store_info=None):
    prompt = create_llm_prompt_for_prediction(features_row, additional_store_info)

    messages = [
        {"role": "system", "content": "You are a sales prediction assistant. Your task is to analyze store sales data and predict future trends."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [6]:
import kagglehub

path = kagglehub.dataset_download("almazkhannanov/liquor-sales")

print("Path to dataset files:", path)

100%|██████████| 61.9M/61.9M [00:03<00:00, 20.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/almazkhannanov/liquor-sales/versions/2


In [7]:
!cp /root/.cache/kagglehub/datasets/almazkhannanov/liquor-sales/versions/2/sazerac_df.csv .
!ls

sample_data  sazerac_df.csv


In [10]:
file_path = 'sazerac_df.csv'

sample_size = 10000
df = load_data(file_path, sample_size=sample_size, random_sample=True)
df = preprocess_data(df)

agg_sales = aggregate_store_product_sales(df)

print("Creating features for prediction...")
features_df = create_store_product_features(agg_sales)

Loading and preprocessing data...
Loaded sample data with shape: (10000, 21)
Sample contains data from 2021-04-01T00:00:00.000 to 2025-01-31T00:00:00.000
Sample includes 1857 unique stores and 37 product categories
Aggregating sales data...
Creating features for prediction...


In [11]:
print("Loading Qwen model...")
model, tokenizer = load_qwen_model()

min_sales = 50
min_periods = 3

Loading Qwen model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Predicting stores with declining sales...


In [12]:
def get_geo_store_info(df, store_id):
    store_data = df[df['store'] == store_id].iloc[0] if any(df['store'] == store_id) else None

    if store_data is not None:
        return f"""
    - Address: {store_data['address']}
    - City: {store_data['city']}
    - Zipcode: {store_data['zipcode']}
    - County: {store_data['county']}
    """
    return None

In [13]:
potential_declining = features_df[
    (features_df['is_declining_basic'] == True) &
    (features_df['avg_sales'] >= min_sales) &
    (features_df['num_periods'] >= min_periods)
]

print(f"Found {len(potential_declining)} store-product combinations with potentially declining sales.")

potential_declining = potential_declining.sort_values('avg_sales', ascending=False)

Found 356 store-product combinations with potentially declining sales.


In [16]:
row = potential_declining.iloc[0]

additional_info = get_geo_store_info(df, row['store_id'])

llm_response = get_llm_prediction(model, tokenizer, row, additional_info)
print(llm_response)

**Prediction: No**
**Confidence: Low**

Based on the provided sales data for the imported vodkas product at the Hy-Vee Food Store in Des Moines from February 1, 2023, through September 1, 2024, there has been no significant decline in sales. The recent three-period average sales have been stable around $2323.16 per bottle, indicating that sales have not declined significantly over the past year. Additionally, the recent change rate is relatively low (-0.94%), suggesting that the trend may be more consistent rather than fluctuating dramatically. Therefore, it seems unlikely that sales will continue to decline further given the current pattern.
